In [1]:
import openai
import json

In [2]:
with open("secrets.json", "r") as f:
    secrets = json.load(f)["key"]
    openai.api_key = secrets

In [3]:
model = "gpt-3.5-turbo"

In [13]:
transcription = """
I want to pick up my prescription, do my laundry, and get some groceries from the grocery store.
"""

messages = [
    {"role": "system", "content": "You are a climate-conscious schedule planner. Your client will give you a list of tasks to complete in a day. Given their schedule, advise them on how to reduce their carbon footprint by changing their schedule. Specifically, you should advise them on things such as combining trips, doing things earlier in the morning to avoid peak hours. Focus on the schedule and not the tasks. Give a concise summary of the optimizations."},
    {"role": "user", "content": "Please list only the order in which I should do the tasks in bullet point form. Here are my tasks: " + str(transcription)},
]

functions = [
{
    "name": "generate_schedule",
    "description": "This function will generate a schedule for the user to view and follow.",
    "parameters": {
        "type": "object",
        "properties": {
            "events": {
                "type": "array",
                "description": "The list of events to be scheduled. Do not format.",
                "items": {
                "type": "object",
                "properties": {
                        "name_of_event": {
                            "type": "string",
                            "description": "The name of the event.",
                        },

                        "description_of_event": {
                            "type": "string",
                            "description": "The description of the event.",
                        },

                        "military_time_interval_of_event": {
                            "type": "string",
                            "description": "The time of the event in military time such as 20:00-21:00. Do not put AM or PM.",
                        },

                        "carbon_points": {
                            "type": "number",
                            "description": "Points that determine how much carbon is emitted by the event. The higher the points, the more carbon is emitted.",
                        },
                    }
                },
            },
        },
    },
}
]

function_call = {
    "name": "generate_schedule",
}


In [14]:
completion = openai.ChatCompletion.create(
    model=model,
    messages=messages,
)

print(completion)

{
  "id": "chatcmpl-827nzxmYWLkh9nEOWveCDSWliYZTw",
  "object": "chat.completion",
  "created": 1695517351,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "- Combine picking up prescription, doing laundry, and getting groceries into one trip to minimize travel.\n- Consider doing these tasks early in the morning or during off-peak hours to avoid the rush and save time."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 137,
    "completion_tokens": 42,
    "total_tokens": 179
  }
}


In [16]:
messages = [
    {"role": "system", "content": "You are a schedule planner. Given a schedule, follow the instructions of the schedule and output a final set of tasks."},
    {"role": "user", "content": 'Please take this schedule and output a list of events: """' + str(completion.choices[0].message) + '"""'},
]

completion = openai.ChatCompletion.create(
    model=model,
    messages=messages,
    functions=functions,
	function_call=function_call,
)

print(completion)

{
  "id": "chatcmpl-827qMqMRVZn5YJTUY5d1NdVZtgkIt",
  "object": "chat.completion",
  "created": 1695517498,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "generate_schedule",
          "arguments": "{\n  \"events\": [\n    {\n      \"name_of_event\": \"Picking up prescription\",\n      \"description_of_event\": \"Go to the pharmacy and pick up prescription.\",\n      \"military_time_interval_of_event\": \"14:00-14:30\",\n      \"carbon_footprint_reduction_by_scheduling\": \"0 kg CO2\"\n    },\n    {\n      \"name_of_event\": \"Doing laundry\",\n      \"description_of_event\": \"Wash and dry clothes.\",\n      \"military_time_interval_of_event\": \"15:00-16:30\",\n      \"carbon_footprint_reduction_by_scheduling\": \"0.5 kg CO2\"\n    },\n    {\n      \"name_of_event\": \"Getting groceries\",\n      \"description_of_event\": \"Go to the grocery 

In [ ]:
event_list = json.loads(completion.choices[0].message.function_call.arguments)['events']

print(f'The transcription """{transcription}""" returns: \n\n')

for event in event_list:
	print(f"The name of the event is {event['name_of_event']}")
	print(f"The description of the event is {event['description_of_event']}")
	print(f"The military time interval of the event is {event['military_time_interval_of_event']}")
	print()

In [ ]:

# change the messages to feed the schedule back into the system and let ChatGPT generate a more optimal schedule
messages = [
    {"role": "system", "content": f"The user has the following schedule:\n\n{json.dumps(event_list, indent=4)}. Rewrite the user's schedule to lower the carbon footprint. Change the order of the events to accomplish this. Do not leave the ordering the same."},
]



In [ ]:

completion = openai.ChatCompletion.create(
    model=model,
    messages=messages,
    functions=functions,
	function_call=function_call
)

print(completion)

In [ ]:
event_list = json.loads(completion.choices[0].message.function_call.arguments)['events']

print(f'The transcription """{transcription}""" returns: \n\n')

for event in event_list:
	print(f"The name of the event is {event['name_of_event']}")
	print(f"The description of the event is {event['description_of_event']}")
	print(f"The military time interval of the event is {event['military_time_interval_of_event']}")
	print()